In [1]:
import pandas as pd
import numpy as np

import plotly.offline as py
import plotly.graph_objs as go

from statsmodels.tsa.arima_model import ARIMA

/usr/local/lib/python3.5/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



In [2]:
df = pd.read_csv('../../train_merged.csv', sep=',', header='infer')

In [3]:
ts = df[(df.Store == 1) & (df.Dept == 17)]
ts = ts[['Weekly_Sales', 'Date']]
ts = ts.set_index('Date')

In [4]:
std =  ts.rolling(window=52).std()
mean = ts.rolling(window=52).mean()
ewma = ts.ewm(halflife=12).mean()

In [5]:
data = [go.Scatter(x=ts.index, y=ts.Weekly_Sales, name='original'),
        go.Scatter(x=std.index, y=std.Weekly_Sales, name='std'),
        go.Scatter(x=mean.index, y=mean.Weekly_Sales, name='mean'),
        go.Scatter(x=ewma.index, y=ewma.Weekly_Sales, name='ewma')]
py.plot(data)

'file:///home/kanoonsantikul/Documents/walmart-sell-predict/src/time-series/temp-plot.html'

In [6]:
ts_diff = ts - mean
ts_diff.dropna(inplace=True)
py.plot([go.Scatter(x=ts_diff.index, y=ts_diff.Weekly_Sales, name='diff'),
        go.Scatter(x=ts.index, y=ts.Weekly_Sales, name='original')])

'file:///home/kanoonsantikul/Documents/walmart-sell-predict/src/time-series/temp-plot.html'

In [7]:
ts_diff = ts - ewma
ts_diff.dropna(inplace=True)
py.plot([go.Scatter(x=ts_diff.index, y=ts_diff.Weekly_Sales, name='diff'),
        go.Scatter(x=ts.index, y=ts.Weekly_Sales, name='original')])

'file:///home/kanoonsantikul/Documents/walmart-sell-predict/src/time-series/temp-plot.html'

In [41]:
model = ARIMA(ts_diff, order=(0, 1, 2))  
results_ARIMA = model.fit()

yhat = results_ARIMA.fittedvalues
yhat *= -1

py.plot([go.Scatter(x=yhat.index, y=yhat.values, name='ARIMA'),
       go.Scatter(x=ts_diff.index, y=ts_diff.Weekly_Sales, name='original')])

'file:///home/kanoonsantikul/Documents/walmart-sell-predict/src/time-series/temp-plot.html'

In [55]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return numpy.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

# load dataset
series = ts
# seasonal difference
X = series.values
weeks_in_year = 52
differenced = difference(X, weeks_in_year)
# fit model
model = ARIMA(differenced, order=(0,1,2))
model_fit = model.fit(disp=0)
# multi-step out-of-sample forecast
start_index = len(differenced)
end_index = start_index + 100
forecast = model_fit.predict(start=start_index, end=end_index)
# invert the differenced forecast to something usable
history = [x for x in X]
inverse = []
for yhat in forecast:
    inverted = inverse_difference(history, yhat, weeks_in_year)
    history.append(inverted)
    inverse.append(inverted[0])
    
py.plot([go.Scatter(y=inverse)])

'file:///home/kanoonsantikul/Documents/walmart-sell-predict/src/time-series/temp-plot.html'